In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import torch
from tqdm.notebook import tqdm

In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = "ashokneupane"
os.environ['KAGGLE_KEY'] = "c8080cb04e61cdb2d2e34d77bc74ad0c"

In [ ]:
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

  0% 0.00/1.99M [00:00<?, ?B/s]
100% 1.99M/1.99M [00:00<00:00, 75.1MB/s]


In [ ]:
!unzip /content/twitter-entity-sentiment-analysis.zip -d /content/

Archive:  /content/twitter-entity-sentiment-analysis.zip
  inflating: /content/twitter_training.csv  
  inflating: /content/twitter_validation.csv  


In [ ]:
from google.colab import drive
drive.mount('content/')

Mounted at content/


# Dataset

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 141.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 100.5 MB/s eta 0:00:00


In [ ]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf


In [ ]:
TRAIN_PATH = '/content/twitter_training.csv'
TEST_PATH = '/content/twitter_validation.csv'

In [ ]:
col_names = ['sn', 'org', 'sentiment', 'text']
train_df = pd.read_csv(TRAIN_PATH, header=None, names=col_names)
test_df = pd.read_csv(TEST_PATH, header=None, names=col_names)

In [ ]:
train_df.tail()

,sn,org,sentiment,text
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...
74681,9200,Nvidia,Positive,Just like the windows partition of my Mac is l...


In [ ]:
test_df.head()

,sn,org,sentiment,text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [ ]:
print(train_df.shape)
print(test_df.shape)

(74682, 4)
(1000, 4)


In [ ]:
# train_df['sentiment'].value_counts()

## label counts
-   -1    22542
-   1    20832
-   0    18318

In [ ]:
label_mapping = {'Positive': 2, 'Neutral': 1, 'Negative': 0}

def preprocessing(df):
  df = df.drop(['sn', 'org'], axis=1)
  df = df[~df['sentiment'].isin(['Irrelevant'])]
  df['sentiment']= df['sentiment'].map(label_mapping)
  df['text'] = df['text'].astype(str)
  return df

In [ ]:
train_df = preprocessing(train_df)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61692 entries, 0 to 74681
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  61692 non-null  int64 
 1   text       61692 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


In [ ]:
test_df = preprocessing(test_df)

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 828 entries, 1 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  828 non-null    int64 
 1   text       828 non-null    object
dtypes: int64(1), object(1)
memory usage: 19.4+ KB


In [ ]:
# import transformers

# seq_len = 256
# batch_size = 16
# num_samples = len(train_df)
# model_name = 'cardiffnlp/twitter-roberta-base-sentiment'

# tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, padding=True, return_tensor='np')

# tokens_train = tokenizer(
#     train_df['text'].tolist(),
#     max_length=seq_len,
#     truncation=True,
#     padding='max_length',
#     add_special_tokens=True,
#     return_tensors='np'
# )



In [ ]:
# num_samples_test = len(test_df)
# model_name = 'cardiffnlp/twitter-roberta-base-sentiment'

# test_tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, padding=True, return_tensor='np')

# tokens_test = tokenizer(
#     test_df['text'].tolist(),
#     max_length=seq_len,
#     truncation=True,
#     padding='max_length',
#     add_special_tokens=True,
#     return_tensors='np'
# )

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# Download the pre-trained GloVe embeddings
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Load the pre-trained GloVe embeddings
glove_file = '/content/content/MyDrive/nlp/glove/glove.6B.100d.txt'

def load_glove_embeddings(glove_file):
    word_vectors = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors

word_vectors = load_glove_embeddings(glove_file)
# Preprocess and tokenize the text
def preprocess_text(text):
    # Tokenize the text
    print(text)
    tokens = word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

# Convert words to word embeddings
def convert_to_word_embeddings(text):
    tokens = preprocess_text(text)

    # Convert each token to its word embedding vector
    embeddings = []
    for token in tokens:
        if token in word_vectors:
            embeddings.append(word_vectors[token])

    return np.array(embeddings)




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
text = "This is an example sentence."
embeddings = convert_to_word_embeddings(text)
embeddings

This is an example sentence.


array([[-0.12617  ,  0.61724  ,  0.22581  ,  0.39868  ,  0.16111  ,
         0.1523   , -0.14715  , -0.29447  , -0.27348  , -0.13753  ,
        -0.20898  , -0.73436  ,  0.14144  ,  0.15048  ,  0.09179  ,
         0.018613 ,  0.22539  ,  0.15979  , -0.16935  ,  0.42716  ,
         0.042284 , -0.3477   , -0.11413  ,  0.12222  , -0.025027 ,
        -0.20805  , -0.067264 , -0.2956   , -0.30807  , -0.32903  ,
         0.19059  ,  0.77141  , -0.19332  , -0.31069  ,  0.26745  ,
         0.32231  ,  0.2065   ,  0.10497  ,  0.49425  , -0.38322  ,
        -0.12802  , -0.069906 , -0.14828  ,  0.085369 , -0.18141  ,
         0.14688  ,  0.60968  , -0.21131  , -0.29148  , -0.52773  ,
         0.59508  ,  0.017369 ,  0.15342  ,  0.81925  , -0.20643  ,
        -2.0378   , -0.11884  , -0.16826  ,  1.5288   ,  0.15756  ,
        -0.4994   ,  0.39305  ,  0.12672  , -0.10968  ,  1.3671   ,
        -0.21006  ,  0.15684  ,  0.0063801,  0.43836  , -0.18765  ,
        -0.29088  ,  0.18619  ,  0.085402 ,  0.1

In [ ]:
# tokens_train

{'input_ids': array([[   0,  757,  562, ...,    1,    1,    1],
       [   0,  100,  524, ...,    1,    1,    1],
       [   0,  757,  562, ...,    1,    1,    1],
       ...,
       [   0, 6785, 5426, ...,    1,    1,    1],
       [   0, 6785, 5426, ...,    1,    1,    1],
       [   0, 6785,  101, ...,    1,    1,    1]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

In [ ]:
# tokens_test

{'input_ids': array([[    0, 28713,   491, ...,     1,     1,     1],
       [    0,  1039, 35460, ...,     1,     1,     1],
       [    0,  6842, 14740, ...,     1,     1,     1],
       ...,
       [    0,  5625, 28635, ...,     1,     1,     1],
       [    0,   387, 12807, ...,     1,     1,     1],
       [    0, 31894,   359, ...,     1,     1,     1]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

In [ ]:
import pandas as pd

# Assuming you have a DataFrame train_df with 'text' column

def convert_to_word_embeddings(text):
    tokens = preprocess_text(text)
    embeddings = []
    for token in tokens:
        if token in word_vectors:
            embeddings.append(word_vectors[token])
    return embeddings

# Apply the function to each value in the 'text' column
train_df['word_embeddings'] = train_df['text'].apply(convert_to_word_embeddings)

im getting on borderlands and i will murder you all ,
I am coming to the borders and I will kill you all,
im getting on borderlands and i will kill you all,
im coming on borderlands and i will murder you all,
im getting on borderlands 2 and i will murder you me all,
im getting into borderlands and i can murder you all,
So I spent a few hours making something for fun. . . If you don't know I am a HUGE @Borderlands fan and Maya is one of my favorite characters. So I decided to make myself a wallpaper for my PC. . Here is the original image versus the creation I made :) Enjoy! pic.twitter.com/mLsI5wf9Jg
So I spent a couple of hours doing something for fun... If you don't know that I'm a huge @ Borderlands fan and Maya is one of my favorite characters, I decided to make a wallpaper for my PC.. Here's the original picture compared to the creation I made:) Have fun! pic.twitter.com / mLsI5wf9Jg
So I spent a few hours doing something for fun... If you don't know I'm a HUGE @ Borderlands fan a

AttributeError: ignored

In [ ]:
y_train = train_df.sentiment.values
labels_train = np.zeros((y_train.size, 3))
labels_train[np.arange(y_train.size), y_train] = 1
labels_train

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [ ]:
y_train

array([2, 2, 2, ..., 2, 2, 2])

In [ ]:
# labels_train.shape

In [ ]:
y_test = test_df.sentiment.values
labels_test = np.zeros((y_test.size, 3))
labels_test[np.arange(y_test.size), y_test] = 1
labels_test

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]])

# Using pretrained model

In [ ]:
# from transformers import pipeline

# sent_pipeline = pipeline("sentiment-analysis")

In [ ]:
# sent_pipeline('I love you')

# Building a transformer

In [ ]:
import tensorflow as tf
import numpy as np

NUM_CLASSES = 3
VOCAB_SIZE = 30000


class Transformer(tf.keras.Model):
    def __init__(self, num_layers, units, d_model, num_heads, dropout, name="transformer"):
        super(Transformer, self).__init__(name=name)
        self.encoder = Encoder(num_layers, units, d_model, num_heads, dropout)
        self.dense = tf.keras.layers.Dense(units)
        self.final_dense = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')

    def call(self, inputs, training=True, mask=None):
        x = self.encoder(inputs, training=training, mask=mask)
        x = tf.reduce_mean(x, axis=1)  # Average pooling over the time dimension
        x = self.dense(x)
        x = tf.nn.relu(x)
        x = self.final_dense(x)
        return x


class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, units, d_model, num_heads, dropout, name="encoder"):
        super(Encoder, self).__init__(name=name)
        self.num_layers = num_layers
        self.units = units
        self.d_model = d_model

        self.embedding = PositionalEmbedding(VOCAB_SIZE, d_model)

        self.enc_layers = [
            EncoderLayer(units, d_model, num_heads, dropout)
            for _ in range(num_layers)
        ]

        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x, training=True, mask=None):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training=training, mask=mask)

        return x


class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, units, d_model, num_heads, dropout, name="encoder_layer"):
        super(EncoderLayer, self).__init__(name=name)
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads, d_model)
        self.ffn = self.point_wise_feed_forward_network(units, d_model)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)

    def call(self, x, training=True, mask=None):
        attn_output = self.mha(x, x, x, mask, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2

    def point_wise_feed_forward_network(self, units, d_model):
        return tf.keras.Sequential(
            [
                tf.keras.layers.Dense(units, activation="relu"),
                tf.keras.layers.Dense(d_model),
            ]
        )


class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model):
        super(PositionalEmbedding, self).__init__()
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
        self.pos_encoding = self.positional_encoding()

    def positional_encoding(self):
        depth = self.d_model // 2
        length = 2048
        positions = np.arange(length)[:, np.newaxis]  # (seq, 1)
        depths = np.arange(depth)[np.newaxis, :] / depth  # (1, depth)

        angle_rates = 1 / (10000 ** depths)  # (1, depth)
        angle_rads = positions * angle_rates  # (pos, depth)

        pos_encoding = np.concatenate(
            [np.sin(angle_rads), np.cos(angle_rads)],
            axis=-1)

        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, x):
        length = tf.shape(x)[1]
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x


In [ ]:
save_path = '/content/content/MyDrive'

num_layers = 6
units = 64
d_model = 300
num_heads = 5
dropout = 0.2
num_epochs = 2
batch_size = 64

model = Transformer(num_layers, units, d_model, num_heads, dropout)
input_seq = tf.keras.Input(shape=(None,))
# mask = tf.keras.Input(shape=(None, None))
# output = model(inputs=input_seq, mask=mask)
output = model(inputs=input_seq)
transformer_model = tf.keras.Model(inputs=input_seq, outputs=output)
# transformer_model = tf.keras.Model(inputs=input_seq, outputs=output)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = f"{save_path}/sentiment_checkpoint.h5",
    save_best_only = True,
    monitor = 'val_loss',
    mode = 'min',
    verbose = 1
)

transformer_model.compile(optimizer=tf.keras.optimizers.Adam(),
                          loss=tf.keras.losses.CategoricalCrossentropy(),
                          metrics=['accuracy'])

# transformer_model.fit(
#     [tokens_train['input_ids'], tokens_train['attention_mask']],
#     labels_train,
#     epochs=10,
#     batch_size=128,
#     validation_data=([tokens_test['input_ids'], tokens_test['attention_mask']], labels_test),
#     callbacks = [checkpoint_callback],
#     verbose = 1
#     )


# transformer_model.fit(
#     tokens_train['input_ids'],
#     labels_train,
#     epochs=10,
#     batch_size=128,
#     validation_data=(tokens_test['input_ids'], labels_test),
#     callbacks = [checkpoint_callback],
#     verbose = 1
#     )

# model.save(f"{save_path}/transformer_sentiment_analysis.h5")
# Convert the text column to a list of strings
texts = train_df['text'].tolist()

# Convert the word embeddings to a NumPy array
embeddings = np.array(train_df['word_embeddings'].tolist())

# Determine the maximum sequence length
max_seq_length = max(len(seq) for seq in embeddings)

# Pad the sequences to the maximum length
padded_embeddings = tf.keras.preprocessing.sequence.pad_sequences(embeddings, maxlen=max_seq_length, padding='post')

# Fit the model using the padded embeddings and labels
transformer_model.fit(padded_embeddings, labels_train, epochs=num_epochs, batch_size=batch_size)


model.save(f"{save_path}/transformer_sentiment_analysis.h5")

KeyError: ignored

## Preditions

In [ ]:
# custom_objects = {'Transformer': Transformer}

# model = tf.keras.models.load_model("/content/content/MyDrive/sentiment_checkpoint.h5",
#                                    custom_objects=custom_objects)


In [ ]:
# import transformers

# def prepare_input_data(text):

#   seq_len = 256
#   model_name = 'cardiffnlp/twitter-roberta-base-sentiment'

#   tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, padding=True, return_tensor='np')

#   tokens = tokenizer(
#       text,
#       max_length=seq_len,
#       truncation=True,
#       padding='max_length',
#       add_special_tokens=True,
#       return_tensors='np'
#   )
#   return [tokens['input_ids'], tokens['attention_mask']]
#   #TODO

In [ ]:
# sentiment_mapping = ['Negaitve', 'Neutral', 'Positive']
# text_data = ["This is a positive review.", "I did not like this product.", "The movie was okay.", "I love you"]
# input_data = prepare_input_data(text_data)

# predictions = transformer_model.predict(input_data)

# predicted_labels = tf.argmax(predictions, axis=1)

# for i, text in enumerate(text_data):
#     print(predicted_labels)
#     print(f"Text: {text}")
#     print(f"Predicted Label: {predicted_labels[i]}")
#     print(f"Sentiment: {sentiment_mapping[predicted_labels[i]]}\n")


In [ ]:
# import sklearn
# from sklearn.metrics import accuracy_score
# y_pred = model.predict([tokens_test['input_ids'], tokens_test['attention_mask']])
# y_pred = tf.argmax(y_pred)
# print(accuracy_score(y_pred, y_test))
